### 要建立預測模型，我們先需要有很多資料
### 由於本次是要透過天氣資料來預測害蟲的數量，因此我們先需要抓取氣象的網頁，並儲存到指定到指定資料夾

#### 在這一部份，我們要先宣告會用到的函數庫
#### random用於抓取間隔時間的抓取，因為如果用常數固定時間休息的話，系統似乎會抓到-------credit(經驗)
#### os 是要確定我們的電腦中是否已經存在某一個文檔，如果有的話，我們就可以不要浪費力氣抓取資料了
#### 使用urllib, requests, socket 來抓取資料，並設定代理人ID
#### 使用csv來讀取檔案，不過存檔是用txt啦!!

In [1]:
import random
import time

import os
import sys #used for passing in the argument

import urllib
import requests
import socket
import urllib.request,urllib.parse,urllib.error

import csv

proxy_support = urllib.request.ProxyHandler({'sock5': 'localhost:1080'})
opener = urllib.request.build_opener(proxy_support)
urllib.request.install_opener(opener)

### 用read_data()來讀取 地區資料 與 東方果實蠅的數量資料
#### 地區資料的資料為: 縣名   鄉鎮市名  測站ID  測站長ID   經度  緯度
#### 蔬果重要害蟲防疫旬報_東方果實蠅數據的資料型態為: 縣名   鄉鎮市名  年  月   旬  本旬東方果實蠅數據  上一旬東方果實蠅數據
##### 由於之後的預測並不會使用道上一旬東方果實蠅的數量數據，所以之後會把它拿掉
##### 讀取成  list of list 的型態

In [2]:
def read_data():
    with open('地區資訊new.csv', 'r') as f:  #opens PW file
        reader = csv.reader(f)
        di = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists
        f.close()

    with open('蔬果重要害蟲防疫旬報_東方果實蠅數據_new.csv','r') as f:
        reader = csv.reader(f)
        df = list(list(rec) for rec in csv.reader(f, delimiter=','))
        #data_fly
        f.close()
    return di, df

###  獲取當天的日期，告訴系統說，資料抓到前一個月就好了，不用繼續抓下去

In [3]:
def today_date():
    import time
    ## dd/mm/yyyy格式
    today_date_year = time.strftime("%Y")
    today_date_month = time.strftime("%m").strip('0')
    today_date_day = time.strftime("%d")
    today_date = today_date_year + today_date_month + today_date_day
    return today_date_year, today_date_month, today_date_day

### 原本是主程式，後來改使用函數帶過去
#### 這是最重要的函式，包含著主要抓取網頁的程式碼
####  資料儲存成filename = 縣名  鄉鎮市區名  年  月.txt  ##(十月以前的月都沒有加上零喔)

In [ ]:
def get_information(row, year, month):
    
    proxy_support = urllib.request.ProxyHandler({'sock5': 'localhost:1080'})
    opener = urllib.request.build_opener(proxy_support)
    urllib.request.install_opener(opener)
    
    if month < 10:
        url ='http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station='+str(row[2])+'&stname='+str(row[5])+'&datepicker='+str(year)+'-0'+str(month)  
    else:
        url ='http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station='+str(row[2])+'&stname='+str(row[5])+'&datepicker='+str(year)+'-'+str(month)  
    time.sleep(random.randint(40,45))
    
    try:
        response = urllib.request.urlopen(url)
        html = response.read()
        errorc=0
        
    except urllib.error.URLError as e:
        if hasattr(e, 'reason'):
            print('We failed to reach a server.')
            print('Reason: ', e.reason)
            errorc=1
            pass
        elif hasattr(e, 'code'):
            print('The server couldn\'t fulfill the request.')
            print('Error code: ', e.code)
            errorc=1
            pass
    else:
        print('everything is fine')
                
    if errorc==0:
        filename = 'C:\\Users\\Aaron\\weather\\' + str(row[0]) + str(row[1]) + str(year) + str(month) + '.txt'
        print(filename)
        with open(filename, 'w') as f:
            f.write(str(html))
            f.close()
        time.sleep(random.randint(4,14))

###  這邊是主程式
### 其步驟如下:
##### 1. 讀取  地區資訊  和  蔬果重要害蟲防疫旬報_東方果實蠅數據  成 list of list 
##### 2. 對於每一個    鄉鎮縣市   年    月   去抓取網頁。其實大可直接   從蔬果重要害蟲防疫旬報_東方果實蠅數據   抓取，但是完成網址url需要測站ID % 長 ID，所以這樣比較方便

In [ ]:
di, df = read_data()
di = di[1:]
df = df[2:]
#data = remove_99999(df)
limitc=0

start = 3
end = 34
for row in di:
    print(row[0],row[1])
    last_url = ''
    for year in [2013,2014,2015,2016]:
        for month in range(1,13):
            a = 0
            filename = 'C:\\Users\\Aaron\\weather\\' + str(row[0]) + str(row[1]) + str(year) + str(month) + '.txt'
            if os.path.isfile(filename):
                a = 1
                continue
            today_date_year, today_date_month, today_date_day = today_date()
            if int(today_date_year) <= year and int(today_date_month) <= month:
                continue
            get_information(row, year, month)
            time.sleep(10.3)
            proxy_support = urllib.request.ProxyHandler({'sock5': 'localhost:1080'})
            opener = urllib.request.build_opener(proxy_support)
            urllib.request.install_opener(opener)
            a = 2
            #C:\Users\Aaron\天氣資料夾
        if a == 2:
            time.sleep(random.randint(4,15))
    if a == 2:
        time.sleep(random.randint(10,25))

台中市 石岡區
台中市 后里區
台中市 和平區
台中市 東勢區
台東縣 台東地區
台南市 大內區
台南市 玉井區
台南市 官田區
台南市 東山區
台南市 南化區
宜蘭縣 冬山鄉
宜蘭縣 員山鄉
宜蘭縣 頭城鎮
花蓮縣 玉溪地區
花蓮縣 花蓮市
花蓮縣 富里鄉
花蓮縣 瑞穗鄉
金門縣 金門地區
南投縣 中寮鄉
南投縣 水里鄉
南投縣 埔里鎮
屏東縣 里港鄉
屏東縣 佳冬鄉
屏東縣 枋山地區
屏東縣 枋寮地區
屏東縣 長治鄉
屏東縣 高樹鄉
屏東縣 麟洛鄉
苗栗縣 三灣鄉
苗栗縣 公館鄉
苗栗縣 卓蘭鎮
桃園市 大溪區
桃園市 復興區
桃園市 龍潭區
高雄市 大社區
高雄市 六龜區
高雄市 田寮區
高雄市 杉林區
高雄市 阿蓮區
高雄市 旗山區
高雄市 燕巢區
雲林縣 斗六市
everything is fine
C:\Users\Aaron\weather\雲林縣斗六市201410.txt
